In [74]:
import requests
from bs4 import BeautifulSoup
# from pathlib import Path
import os
from datetime import datetime
from glob import glob
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import pyperclip
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait




file_list = glob(r'C:\Users\inyoung\Desktop\git_programmers\*')
file_create_date = {}
now_date_upload_llist = []
folders_file = []
# 폴더 안의 파일 모두 보기
for file in file_list:
    if os.path.isdir(file):
        folders_file = glob(file+'\*')
        # print(file)

# 파일 생성 날짜 딕셔너리 만들기
for folder in folders_file:
    # print(folder)
    file_create_date[folder[folder.index('0\\')+2:]] = datetime.fromtimestamp(os.path.getctime(folder)).strftime('%Y-%m-%d')
    file_create_date[folder[folder.index('0\\')+2:]] = [datetime.fromtimestamp(os.path.getctime(folder)).strftime('%Y-%m-%d'), folder]

now = datetime.now().strftime('%Y-%m-%d')
now_date_upload_file = []
now_date_upload_list = []
# print(now)
for name, date in file_create_date.items():
    if date[0] == now:
        now_date_upload_list.append(name)
        now_date_upload_file.append(date[1])

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

driver.get('https://velog.io/')
time.sleep(1)
# 로그인 버튼
driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[2]/button[2]').click()
time.sleep(.5)

# 깃허브 선택
driver.find_element(By.CSS_SELECTOR, '#root > div.sc-gsDKAQ.dWETBP > div > div.white-block > div.block-content > div > div.upper-wrapper > section:nth-child(3) > div > a:nth-child(1)').click()
time.sleep(.5)
# 아이디 비밀번호 입력, 로그인
driver.find_element(By.CSS_SELECTOR, '#login_field').send_keys('dlsdud9098@naver.com')
driver.find_element(By.CSS_SELECTOR, '#password').send_keys('dud7959098@')
driver.find_element(By.CSS_SELECTOR, '#login > div.auth-form-body.mt-3 > form > div > input.btn.btn-primary.btn-block.js-sign-in-button').click()

time.sleep(.5)

for idx, file in zip(now_date_upload_list, now_date_upload_file):
    # rq = requests.get(f'https://school.programmers.co.kr/learn/courses/30/lessons/{파일 이름}')
    rq = requests.get('https://school.programmers.co.kr/learn/courses/30/lessons/120862')

    with open(file, 'r') as file:
        code = file.read()

    soup = BeautifulSoup(rq.content, 'html.parser')

    result = []
    title = soup.select_one('#tab > li').text.lstrip().rstrip()
    content = str(soup.select_one('#tour2 > div'))
    content = content.split('\n')
    for con in content:
        con = con.replace('h6', 'h2')
        con = con.replace('h5', 'h2')
        if '문제 설명' in con:
            con = con.replace('문제 설명', '💡문제 설명')
        if '제한사항' in con:
            con = con.replace('제한사항', '🚫제한사항')
        if '입출력 예' in con:
            con = con.replace('<h2>입출력 예', '<h2>🔢입출력 예')
        if '입출력 예 설명' in con:
            con = con.replace('🔢입출력 예 설명', '🔍입출력 예 설명')

        result.append(con)

    result.append('<hr>')
    result.append('<h2>💻코드</h2>')
    result.append('\n')
    result.append(f'''
```python
{code}
```
    ''')

    # result
    velog_content = []
    velog_content_all = ''.join(result)

    # velog

    # 글쓰기 버튼
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[1]/div/div[2]/button[2]').click()

    # 제목 쓰기                          
    ele = driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[1]/div/div[1]/div[1]/div/textarea')
    ele.send_keys('프로그래머스 '+title)

    # 태그
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div/div[1]/div/div[1]/div[1]/div/div[2]/input').send_keys('프로그래머스, 파이썬,')

    # 내용 쓰기
    ele = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div/div[1]/div/div[1]/div[3]/div/div[6]/div[1]/div/div/div/div[5]').click()
    act = ActionChains(driver)

    pyperclip.copy(velog_content_all)
    act.key_down(Keys.CONTROL).send_keys("v").perform()

    time.sleep(1)
    # 출간하기 버튼
    try:
        driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div/div[1]/div/div[2]/div/div/button[2]').click()
    except NoSuchElementException:
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '#root > div.sc-gLDmcm.giPzuI > div > div.sc-ehCJOs.auvDf > div > div.sc-eLwHnm.BNgcW > div > div > button.sc-jrQzAO.jYsOEX.sc-fvxzrP.hiArGR').click()
    time.sleep(1)

    # 시리즈 선택
    try:
        driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div[3]/div[1]/section[3]/div/button').click()
    except NoSuchElementException:
        driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div[3]/div[1]/section[3]/div/button').click()
    time.sleep(1)
    # 시리즈 선택
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div[3]/section/div/div[1]/ul/li[5]').click()
    time.sleep(1)
    # 선택하기
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div[3]/section/div/div[2]/button[2]').click()
    time.sleep(.5)
    # 출간하기
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div[3]/div[2]/button[2]').click()



    driver.close()

C:\Users\inyoung\AppData\Local\Temp\ipykernel_19312\3874536191.py:47: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
